# Natural Gradient

### Table of Contents

* [1. Theory](#chapter1)
    * [1.1 Classical natural gradient](#section_1_1)
    * [1.2 Quantum natural gradient](#section_1_2)
    * [1.3 Code implementation problems](#section_1_3)
    * [1.4 Code implementation solutions](#section_1_4)
* [2. Code](#chapter2)
    * [2.I Parameters](#section_2_I)
    * [2.II Imports](#section_2_II)
    * [2.1 Replay Memory](#section_2_1)
    * [2.2 Quantum Variational Circuit](#section_2_2)
    * [2.3 Agent](#section_2_3)
    * [2.4 Main](#section_2_4)
* [3. Comparison code](#chapter3)

### 1. Theory <a class="anchor" id="chapter1"></a>

#### 1.1 Classical natural gradient  <a class="anchor" id="section_1_1"></a> 

A neural netowork (or QVC, in our case), is a function approximator. 
Training a neural network means finding the parameters that best bring it closer to the unkwnown function that theoretically perfectly solves the task.
In doing so, it makes sense to work in the space of distributions. In the In order to measure the distance between two distributions we have to introduce a metric. A common one is the KL divergence. So we define the distance between two distributions as:

$$
    D_{KL}(f|g) = \sum\limits_{x \in X} f(x) \log\left( \frac{f(x)}{g(x)} \right)
$$

where, in the case of neural networks, we can interpret $f$ as the function approximator, $g$ the function we want it to approach, and $X$ out training data.

Once we introduce a new metric in a space, this leads to a different notion of distance. 
In fact we have two different spaces:
1) The space where the functions leave, and where the metric is the KL-divergence </br>
2) The space where the representation is coordinates of the functions leave, and where the metric is the euclidean one, so the identity matrix.

For example, if we consider a gaussian distribution of mean $\mu$ and standard deviation $\sigma$, then we can represent a gaussian as the point $(\mu, \sigma) \in \mathbb{R}^2$. 

Now let's say we have two gaussians, $\mathcal{N_1}$ and $\mathcal{N_2}$, and we represent them as two vectors in $\mathbb{R}^2$: $\vec{n}_1 = (\mu_1, \sigma_1)$ and $\vec{n}_2 = (\mu_2, \sigma_2)$. Then we can calculate the distance between these two gaussians with the standard euclidean way: 

$$
D_{Euclidean}(\mathcal{N_1}|\mathcal{N_2}) = \|\vec{n}_1 - \vec{n}_2\| = \sqrt{(\vec{n}_1 - \vec{n}_2)\cdot(\vec{n}_1 - \vec{n}_2)} = \sqrt{(\mu_1 - \mu_2)^2 + (\sigma_1 - \sigma_2)^2}
$$

This is basically what we do when we do the standard optimization methods: we consider two functions, the function approximator $f$ depending on the parameters $\theta_f$ and the theoretical unknown function $g$, depending on the parameters $\theta_g$, we calculate the gradient in the space of the parameters, and then we update them, so that $f$ gets closer to $g$. Getting closer means thoguh getting closer in the sense of the euclidean metric for the parameters. We are saying that at each training step $\|\theta_f - \theta_g\|$ gets smaller.

Now we remember thoguht that the functions leave not in the euclidean space, where we resemble them with the parameters. They belong instead in a space where we set a different notion of distance. With the example of the two gaussians above, the distance would be:

$$
    D_{KL}(\mathcal{N_1}|\mathcal{N_2}) = \sum\limits_{x \in X} \mathcal{N_1}(x) \log\left( \frac{\mathcal{N_1}(x)}{\mathcal{N_1}(x)} \right)
$$

So the question now is: provided that we have a new notion of distance, how should we move in the space of the parameters $(\mu, \sigma)$, which is the only thing we can control, so that the $D_{KL}$ gets smaller, and not the $D_{Euclidean}$?

From the equations above we can see that the notion of distance depends on the scalar product. We measure the lenghts of a vector as, ignoring the square root:

$$
    \|\vec{v}\|^2 = \vec{v} \cdot \vec{v} = \sum\limits_{i = 1}^N v^2_i = \sum\limits_{i = 1}^N v_i \mathbb{1}_{ij} v_j = \vec{v}^t \mathbb{1} \vec{v}
$$

where, in the last equation, we introduced explicitly the identity matrix.

The point is to understand which matrix we should use to measure the distance with respect to the $D_{KL}$ insted of the euclidean one, so which matrix should take the place of the identity one in the above equation.

In can be proved [1] that the matrix that naturally comes up expanding the $D_{KL}$ up to the second order is the Fisher information matrix [2], which has the form:

$$
    F = \mathbb{E}_{x}\left[ \nabla_{\theta}f(x) \nabla_{\theta}f(x)^t \right]
$$

More specifically it can be proved that 

$$
    D_{KL}(f_{\theta} | f_{\theta + d}) \approx \frac{1}{2}d^t F d
$$

which resembles the equation above, where now $\vec{d}$ is the vector inside the scalar product, and we see that the matrix that appears naturally, and that represents the metric tensor, is the fisher information matrix.

We now discovered that, if we want to move in the space of parameters in such a way that the $D_{KL}(f|g)$ gets smaller, and not the euclidean one, then we have to measure the distances using the Fisher information matrix. 

How does it affect the steps we are taking with the gradient? It can be proved [1] that the gradient gets affected in the following way: if we call $\tilde{\nabla}_{\theta}f$ the gradient that takes into account the new notion of distance, then

$$
    \tilde{\nabla}_{\theta}f = F^{-1}\nabla_{\theta}f 
$$


References part 1: </br>
[1] https://agustinus.kristia.de/techblog/2018/03/14/natural-gradient/ </br>
[2] https://agustinus.kristia.de/techblog/2018/03/11/fisher-information/ </br>
[3] https://deep-and-shallow.com/2020/04/01/the-gradient-boosters-via-natural-gradient/

#### 1.2 Quantum natural gradient  <a class="anchor" id="section_1_2"></a> 

Now we have to understand how to translate the Fisher information matrix in the quantum case. A quantum analogue of the Fisher information matrix, which reduces to the classical one in the classical limit, has been proposed [1]. This is called the Fubini-Study metric, and we call it $g$. There is a good explanation in a pennylane tutorial [2].

The Fubini-Study metric is defined as follows: let's consider a quantum circuit made of parametrised gates, which we will call $V(\theta)$ and unparametrised gates, which we will call $W$. Then the whole circuit $U(\theta)$ is:

$$
    U(\theta)|\psi_0> = V_{L}(\theta_{L})W_{L} V_{L-1}(\theta_{L-1})W_{L-1} \dots V_{0}(\theta_{0})W_{0} |\psi_0>
$$

for each parametric layer $l$ in the variational quantum circuit the $n_l \times n_l$ block diagonal submatrix of the Fubini-Study tensor $g^{(l)}_{ij}$ is

$$
    g^{(l)}_{ij} = <\psi_{l-1}|K_i K_j|\psi_{l-1}> - <\psi_{l-1}|K_i|\psi_{l-1}><\psi_{l-1}|K_j|\psi_{l-1}>
$$

where 

$$
    |\psi_{l-1}> = V_{l-1}(\theta_{l-1})W_{l-1} \dots V_{0}(\theta_{0})W_{0} |\psi_0>
$$


References part 2: </br>
[1] https://arxiv.org/pdf/1909.02108.pdf </br>
[2] https://pennylane.ai/qml/demos/tutorial_quantum_natural_gradient.html

#### 1.3 Code implementation problems  <a class="anchor" id="section_1_3"></a> 

We can use the natural gradient in pennylane simply by selecting the Qunatum Natural Gradient optimizer. 

As an example, let's use the one given by the pennylane tutorial mentioned above. 

In [ ]:
# imports
import pennylane as qml
from pennylane import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# circuit definition
dev = qml.device("default.qubit", wires=3)

@qml.qnode(dev)
def circuit(params):
    # |psi_0>: state preparation
    qml.RY(np.pi / 4, wires=0)
    qml.RY(np.pi / 3, wires=1)
    qml.RY(np.pi / 7, wires=2)

    # V0(theta0, theta1): Parametrized layer 0
    qml.RZ(params[0], wires=0)
    qml.RZ(params[1], wires=1)

    # W1: non-parametrized gates
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])

    # V_1(theta2, theta3): Parametrized layer 1
    qml.RY(params[2], wires=1)
    qml.RX(params[3], wires=2)

    # W2: non-parametrized gates
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])


    return qml.expval(qml.PauliY(0))

And now we run a circuit with a classical gradient and a quantum natural gradient

In [ ]:
# initial parameters
steps = 200
init_params = np.array([0.432, -0.123, 0.543, 0.233], requires_grad=True)


# quanntum natural gradient cost and optimizer
qng_cost = []
opt = qml.QNGOptimizer(0.01)

# training loop with quantum natural gradient
theta = init_params
for _ in range(steps):
    theta = opt.step(circuit, theta)
    qng_cost.append(circuit(theta))
    
# classical gradient cost and optimizer
gd_cost = []
opt = qml.GradientDescentOptimizer(0.01)

#training loop with classical gradient
theta = init_params
for _ in range(steps):
    theta = opt.step(circuit, theta)
    gd_cost.append(circuit(theta))
    

# plot results
plt.style.use("seaborn")
plt.plot(gd_cost, "b", label="Vanilla gradient descent")
plt.plot(qng_cost, "g", label="Quantum natural gradient descent")

plt.ylabel("Cost function value")
plt.xlabel("Optimization steps")
plt.legend()
plt.show()

What is confusing is the fact that the function `opt.step()` wants, as a first argument, the objective function. Which means that here the circuit itself is passed as an objective function. So the algorithm here is simply trying to minimize the output of the circuit itself. 

So let's try to set a different cost function, and then run the same code:

In [ ]:
# cost function = MSE between the output of the circuit and 0
def cost(theta):
    loss = (circuit(theta))**2
    return loss

# quanntum natural gradient cost and optimizer
qng_cost = []
opt = qml.QNGOptimizer(0.01)

# training loop with quantum natural gradient
theta = init_params
for _ in range(steps):
    theta = opt.step(cost, theta)
    qng_cost.append(circuit(theta))

So the problem seems to be that pennylane can automatically calculate the quantum natural gradient, thus the metric tensor, only if the objective function is a pure quantum node, so if no mathematical manipulation is involved. 

#### 1.4 Code implementation solutions  <a class="anchor" id="section_1_4"></a> 

In fact, looking online, I found the following [1] discussion where they talk about the same problem:

Here the pennylane developer says that, altohugh there is not a ready implemented way to calculate the natural gradient automatically, we can derive manually the mathematical part of the cost function, and then, when it comes to deriving the circuit, adding the metric tensor. 

For example, in our case, the loss function without target net is:

$$
    L = \mathbb{E}_{a, s, s'} \left[ \left(r + \gamma \max Q(s', a;\theta) - Q(s, a;\theta)\right)^2 \right]
$$

If we derive it with the natural gradient $\tilde{\nabla}_{\theta}$:

$$
    \tilde{\nabla}_{\theta}L =  \tilde{\nabla}_{\theta} \mathbb{E} \left[ \left(r + \gamma \max Q(s', a;\theta) - Q(s, a;\theta)\right)^2 \right] = \mathbb{E} \left[ \tilde{\nabla}_{\theta}  \left(r + \gamma \max Q' - Q\right)^2 \right] = \dots
$$

where we used the fact that can be shown [2] that the gradient can be brought inside the integral of the expected value.

Also, from now on I will call $Q' := Q(s', a;\theta)$ and $Q := Q(s, a;\theta)$. 

$$ 
    \dots = \mathbb{E} \left[ 2 \left(r + \gamma \max Q' - Q\right) \cdot  \tilde{\nabla}_{\theta} \left(r + \gamma \max Q' - Q\right) \right] = \mathbb{E} \left[ 2 \left(r + \gamma \max Q' - Q\right) \cdot \left(\gamma \tilde{\nabla}_{\theta} \left(\max Q'\right) - \tilde{\nabla}_{\theta}Q \right) \right] = \dots
$$

Let's write (see Appendix for the explanation) now more explicitly the following derivative $\tilde{\nabla}_{\theta} \left(\max Q'\right)$.

$$
    \tilde{\nabla}_{\theta} \left(\max Q'\right) = g^{-1} \nabla_{\theta}\left(\max Q'\right) = g^{-1} \cdot \left(J_{\theta} Q'(\theta) \right)^t \cdot \nabla_{Q'} max(Q')
$$

So, inserting this back into the equation:

$$
 \dots = \mathbb{E} \left[ 2 \left(r + \gamma \max Q' - Q\right) \cdot \left(\gamma \cdot g^{-1} \cdot \left(J_{\theta} Q'(\theta) \right)^t \cdot \nabla_{Q'} max(Q')  - \tilde{\nabla}_{\theta}Q \right) \right]
$$



Now I had to figure out how to calcualte the derivative of the $\max$ function, which I didn't know. Analitically it seems to be a pretty hard result to be implemented [3]. But then I thought that, when we do reinforcement learning with classical neural networks, we already have this function as the loss function, which pytorch can derive during the backpropagation. So I thought that I only had to try to understand in which way pytorch backpropagates thorough the max function. Apparently [4] pytorch treats the max function as an identity of the max value, thus deriving it giving a 1 on the max value selected, 0 otherwise, so that

$$
    \nabla_{Q'} max(Q') = \begin{pmatrix} 1 \\ 0 \end{pmatrix} \text{  if  $Q_1$ is chosen, } \begin{pmatrix} 0 \\ 1 \end{pmatrix} \text { otherwise }
$$

In conclusion

$$
    \tilde{\nabla}_{\theta}L = \mathbb{E} \left[ 2 \left(r + \gamma \max Q' - Q\right) \cdot \left(\gamma \cdot g^{-1} \cdot \left(J_{\theta} Q'(\theta) \right)^t \cdot \nabla_{Q'} max(Q')  - g^{-1} \cdot \nabla_{\theta}Q \right) \right]
$$

Comments:
- g is not in general a constant matrix, it depends on the state since it's evaluated at each point. That's the reason why we cannot group it from the parenthesis, writing only one $g^{-1} ( \dots)$. In fact, more explicetly, it would be
$$
    \tilde{\nabla}_{\theta}L = \mathbb{E} \left[ 2 \left(r + \gamma \max Q' - Q\right) \cdot \left(\gamma \cdot g^{-1}(s') \cdot \left(J_{\theta} Q'(\theta) \right)^t \cdot \nabla_{Q'} max(Q')  - g^{-1}(s) \cdot \nabla_{\theta}Q \right) \right]
$$

- We keep in mind that $Q(s, a; \theta)$ is the function representing our QVC, so the one that in the discussion [1] or in the  above section is represented by $U(\cdot)$, so basically what we have to do is to calculate $g$ manually, tell pennylane to classically derive the circuit and then put them together. 

Appendix: 

We call $\mathbb{R}^n$ the space of parameters and $\mathbb{R}^m$ the action space. Then, seeing $Q$ as a function of $\theta$, we have $Q : \mathbb{R}^n \to \mathbb{R}^m$ and $max: \mathbb{R}^m \to \mathbb{R}$

We note that we treat the gradients as column vectors $(nx1)$, while actually they would be line vectors $(1xn)$, but since we want to apply a matrix $A = g^{-1}$ on the left, so $A \nabla_{\theta}f$, then we have to consider $\nabla_{\theta} f$ as a column vector. Otherwise we should do $\nabla_{\theta}f A^{t}$. 

In fact, treating the gradient as it should, namely a line vector $(1xn)$, we would have($-t = -1$ and $t$)

$$
\tilde{\nabla}_{\theta} max(Q')  = \nabla_{\theta} max(Q') g^{-t} = \nabla_{Q'}max(Q') \cdot J_{\theta}Q \cdot g^{-t}
$$

in this way the dimensions match: $ (1 \times n) = (1 \times n) \times (n \times n) = (1 \times m) \times (m \times n) \times (n \times n)$.

Alternatively, we can treat the gradient as a column vector. In that case:

$$
\tilde{\nabla}_{\theta} max(Q')  = g^{-1} \nabla_{\theta} max(Q')  = \cdot g^{-1} \cdot (J_{\theta}Q)^t \cdot  \nabla_{Q'}max(Q') 
$$

Also like this the dimensions match:$(n \times 1) = (n \times n) \times (n \times 1) = (n \times n) \times (n \times m) \times (m \times 1)$

References part 4: </br>
[1] https://discuss.pennylane.ai/t/variational-classifiers-and-qngoptimizer/524/2 </br>
[2] https://ai.stackexchange.com/questions/14321/how-is-the-gradient-of-the-loss-function-in-dqn-derived </br>
[3] https://math.stackexchange.com/questions/1237239/what-is-the-derivative-of-max-and-min-functions </br>
[4] https://stackoverflow.com/questions/53539348/what-is-the-backward-process-of-max-operation-in-deep-learning


## 2. Code <a class="anchor" id="chapter2"></a>

### 2.I. Parameters <a class="anchor" id="section_2_I"></a>

In [1]:
NUM_QBITS = 4
NUM_LAYERS = 2
LR = 0.01
BATCH_SIZE = 64
NUM_LEARNING_ITERATIONS = 1
SHOW_CIRCUIT = False
USING_BIASES = False
APPLY_SOFTMAX = False

### 2.II. Imports <a class="anchor" id="section_2_II"></a>

In [2]:
# usual
import numpy as np
import matplotlib.pyplot as plt
from pennylane import numpy as np_pl

# scipy
import scipy as sp

# pennylane and gym
import pennylane as qml
import gym

# torch
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torch.optim as optim

# tensorboard
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

/Users/alessandro/opt/anaconda3/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
/Users/alessandro/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


### 2.1. Replay Memory <a class="anchor" id="section_2_1"></a>

In [3]:
class ReplayMemory():
    """
    Handmade replay memory for the agent
    """
    def __init__(self, max_mem_size, input_dim):
        # maximum size of memory
        self.mem_size = max_mem_size
        
        # counter of how much info in the memory
        self.mem_counter = 0
        
        # information to be stored
        self.state_memory = np.zeros((self.mem_size, *input_dim), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_dim), dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.bool)
    
    def push(self, state, action, reward, next_state, done):
        """
        push new information into the memory
        """
        # index to eventually overwrite past memory
        index = self.mem_counter % self.mem_size
        
        # push the information
        self.state_memory[index] = state
        self.new_state_memory[index] = next_state
        self.reward_memory[index] = reward
        self.action_memory[index] = action
        self.terminal_memory[index] = done
        
        # increase the counter
        self.mem_counter += 1
        
    def sample(self, batch_size):
        """
        get some information from the memory
        """
        # get number of info in memory
        max_mem = min(self.mem_counter, self.mem_size)
        
        # select 'batch_size' randon numbers in the interval [0, max_mem]
        sample_indices = np.random.choice(a = max_mem, size = batch_size, replace=False)
        
        # get an array of evenly spaced ints [0, 1, 2,..., batch_size]
        batch_indices = np.arange(batch_size, dtype=np.int32)
        
        # sample from the memory 
        state_batch = np_pl.tensor(self.state_memory[sample_indices])
        new_state_batch = np_pl.tensor(self.new_state_memory[sample_indices])
        reward_batch = np_pl.tensor(self.reward_memory[sample_indices])
        terminal_batch = np_pl.tensor(self.terminal_memory[sample_indices])
        action_batch = self.action_memory[sample_indices]
        
        return batch_indices, state_batch, new_state_batch, reward_batch, terminal_batch, action_batch 

### 2.2 Quantum Variational Circuit <a class="anchor" id="section_2_2"></a>

In [4]:
# helper functions used later in the QVC class

# rotation operators
def rotations(w_layer):
    for i, w_wire in enumerate(w_layer):
        # 3D rotation
        theta_x, theta_y, theta_z = w_wire
        qml.Rot(theta_x, theta_y, theta_z, wires=i)
        
# entangling operators 
def entangling(n_qubits):
    for i in range(n_qubits-1):
        qml.CNOT(wires=[i,i+1])
        
    qml.CNOT(wires=[n_qubits-1, 0])

# encoding the observations
def encode(state):
    for i, s in enumerate(state):
        # not very elegant this try/except 
        try:
            qml.RX(np.arctan(s), wires=i)
            
        except:
            s = s._value
            qml.RX(np.arctan(s), wires=i)

In [15]:
class QVC(nn.Module):
    """
    Quantum Variational Circuit with a pytorch interface 
    """
    def __init__(self, num_layers, num_qubits, lr, action_space):
        super().__init__()
        
        # layers and qubits
        self.num_layers = num_layers
        self.num_qubits = num_qubits
        
        # action space dim
        self.action_space = action_space
        
        # pennylane device used to simulate the circuit
        self.device = qml.device('default.qubit', wires=num_qubits)
        # pennylane quantum node
        self.qnode = qml.QNode(self.circuit, self.device)
    
        # parameters
        w = np_pl.random.random(size=(self.num_layers, self.num_qubits, 3), requires_grad=True)
        w = w*10 - 5
        w = 2*np_pl.arctan(w)                
        self.weights = w
        
        # optimizer
        self.optimizer = qml.AdamOptimizer(0.01)
        
        # optionals
        self.using_biases = USING_BIASES 
        self.apply_softmax = APPLY_SOFTMAX
        
    def circuit(self, weights, states):
        """
        Create the circuit
        """
        # encoding
        encode(states)
            
        # for every layer
        for i in range(self.num_layers):
            # CNOTs
            entangling(self.num_qubits)
            
            # ROTs
            rotations(weights[i])
                
        return [qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(1))]
        
        
    def forward(self, states_batch):
        """ 
        Forward function 
        """
        # call the circuit for every state in the batch
        measurements = np_pl.stack([self.qnode(self.weights, state) for state in states_batch])
        
        # transofrm from numpy tensor to pennylane numpy tensor 
        measurements = np_pl.array(measurements, requires_grad=True)
        
        print(measurements.shape, type(measurements))
                
        # apply biases
        if self.using_biases:
            measurements = measurements + self.biases
                
        return measurements
        
        
    def show(self, state):
        """
        Print circuit architecture
        """
        drawer = qml.draw(self.qnode)
        print(drawer(self.weights, state))

### 2.3. Agent <a class="anchor" id="section_2_3"></a>

In [18]:
class Agent():
    """
    Agent that will implement the deep Q-Learning algorithm
    """
    def __init__(self, gamma, epsilon, lr, input_dims, batch_size, n_actions, max_mem_size = 100000, eps_end=0.01, eps_dec=5e-4):
        # agent params
        self.epsilon = epsilon
        self.eps_min = eps_end
        self.eps_dec = eps_dec
        self.lr = lr
        self.batch_size = batch_size
        self.gamma = gamma
        
        # action space
        self.action_space = [i for i in range(n_actions)]
        
        # neural net
        self.neural_net = QVC(num_layers=NUM_LAYERS, num_qubits=NUM_QBITS, lr=self.lr, action_space=self.action_space)
        
        # memory
        self.memory = ReplayMemory(max_mem_size, input_dims)
            
    
    def store_transition(self, state, action, reward, state_, done):
        """
        Store one transition into the memory
        """
        self.memory.push(state, action, reward, state_, done)
        
        
    def choose_action(self, observation):
        """
        Choose next action
        """
        # if epsilon is low enough 
        if np.random.random() > self.epsilon:
            # then choose with the neural net
            state = np_pl.tensor([observation])
            actions = self.neural_net.forward(state)
            action = np_pl.argmax(actions).item()
        else:
            # otherwise do a random action
            action = np.random.choice(self.action_space)
        
        return action
 

    def quantum_natural_gradient(self):
        """
        Calculate and return the quantum natural gradient
        """
    
        # sample some information from the memory
        batch_indices, state_batch, new_state_batch, reward_batch, terminal_batch, action_batch = self.memory.sample(self.batch_size)
    
        # calculate Q(s,a), the Q value of the initial state
        Q_itial_state = self.neural_net.forward(state_batch)[batch_indices, action_batch]
        
        # calculate Q(s',a), the Q value of the final state
        Q_final_state = self.neural_net.forward(new_state_batch)
        # if the final state was a terminal one, then there is no expected value for future moves
        Q_final_state[terminal_batch] = 0.0
        # calculate Y = r + γmax(Q(s',a))
        Y = reward_batch + self.gamma * np_pl.max(Q_final_state)
        
        
        # redefine weights and circuit
        weights = self.neural_net.weights
        circuit = self.neural_net.qnode
        # pennylane object that is able to calculate the gradient of a circuit
        quantum_grad = qml.grad(circuit) 
        
        
        # quantum natural gradient evalued in present state
        quantum_nat_grad_present_state = np.empty((BATCH_SIZE, NUM_LAYERS, NUM_QBITS, 3))
        quantum_nat_grad_next_state = np.empty((BATCH_SIZE, NUM_LAYERS, NUM_QBITS, 3))
        
        
        # compute gradient of Q evaluated in present state
        for i, x in enumerate(state_batch):
            x = np.array(x, dtype=np.float64)
            
            print(x.dtype, x)
            print(weights.dtype, weights)
            
            # compute the classical gradient of each QNode with repsect to `weights`
            classical_grad_present_state = quantum_grad(weights, x)
        
            # compute the metric tensor for the present state
            num_params = np.prod(classical_grad_present_state.shape)
            g = circuit.metric_tensor([weights, x])
            
            # invert metric tensor
            g_inverse = np.linalg.inv(g) 
    
            # apply g^-1 to the classical gradient
            quantum_nat_grad_present_state[i] = np.linalg.solve(g_inverse, classical_grad_present_state.flatten()).reshape(*classical_grad_present_state.shape) 
            
            
        # compute gradient of Q evaluated in next state
        for i, x in enumerate(new_state_batch):
            x = np.array(x, dtype=np.float64)
            
            # compute the classical gradient of each QNode with repsect to `weights`
            classical_grad_next_state = quantum_grad(weights, x)
            
            # compute the metric tensor for the next state
            num_params = np.prod(classical_grad_next_state.shape)
            g = circuit.metric_tensor([weights, x])
            
            # invert metric tensor
            g_inverse = np.linalg.inv(g)
    
            # apply g^-1 to the classical gradient
            quantum_nat_grad_next_state[i] = np.linalg.solve(g_inverse, classical_grad_next_state.flatten()).reshape(*classical_grad_next_state.shape)
            
        
        # calculate the gradient
        natural_gad = 2*( (Y - Q_itial_state)*(quantum_nat_grad_next_state - quantum_nat_grad_present_state) ).mean()
            
        return natural_grad
        
        
    def learn(self):
        """
        Learn from past experience
        """
        # if there is not enough information in the memory
        if self.memory.mem_counter < self.batch_size:
            # it's not yet time to learn
            return
        
        # otherwise, for a certain amount of iterations
        for _ in range(NUM_LEARNING_ITERATIONS):
            self.neural_net.weights += self.quantum_natural_gradient()
            
        # decrease epsilon
        if self.epsilon > self.eps_min:
            self.epsilon = self.epsilon - self.eps_dec
        else:
            self.epsilon = self.eps_min

### 2.4. Main <a class="anchor" id="section_2_4"></a>

In [19]:
# crate environment 
env = gym.make('CartPole-v1')

# observation and action space
n_actions = env.action_space.n
input_dim = env.observation_space.shape[0]

# create agent
agent = Agent(gamma=0.99, epsilon=1, batch_size=64, n_actions=n_actions, eps_end=0.01, input_dims=[input_dim], lr=0.0001)

# params
scores, eps_history = [], []
n_games = 500

# for every episode
for i in range(n_games):
    # reset everything
    score = 0
    cost = 0
    done = False
    observation = env.reset()
    grad_norm_weights_episode = []
    grad_norm_biases_episode = []
    
    # while the agent is still playing
    while not done:
        
        # choose an action
        action = agent.choose_action(observation)
                
        # make a step 
        new_observation, reward, done, info = env.step(action)
            
        # sum the reward
        score += reward
                
        # store the transition 
        agent.store_transition(observation, action, reward, new_observation, done)
                
        # learn from experience
        agent.learn()
        
        # start from new observation
        observation = new_observation
        
    # append the score 
    scores.append(score)

    # append epsilon
    eps_history.append(agent.epsilon)
    
    # avrage the score 
    avg_score = np.mean(scores[-50:])
    
    # tensorboard writer
    writer.add_scalar("Score", score, i)
    writer.add_scalar("Epsilon", agent.epsilon, i)

    # print information every x episodes
    if i%10 == 0:
        print('episode', i, 'score %.2f' % score, 'last 10 average score %2.f' % avg_score, 'epsilon %.2f' %agent.epsilon)
        if SHOW_CIRCUIT:
            agent.neural_net.show(observation)

episode 0 score 17.00 last 10 average score 17 epsilon 1.00
(64, 2) <class 'pennylane.numpy.tensor.tensor'>
(64, 2) <class 'pennylane.numpy.tensor.tensor'>
float64 [-0.15728334 -0.98501927  0.15781839  1.53927314]
float64 [[[ 1.95225946 -2.376468   -2.57398311]
  [ 1.74578449 -2.56262423  0.19412843]
  [-0.73551876  1.79267786 -2.67365615]
  [-0.51492617 -2.49155222 -1.3299938 ]]

 [[ 2.51202024  0.37888621  2.27847373]
  [ 1.63568513  2.6887829  -2.48172611]
  [-2.59458479 -1.7786416  -2.61569145]
  [ 2.57618011  2.22706786 -2.55062069]]]


TypeError: Grad only applies to real scalar-output functions. Try jacobian, elementwise_grad or holomorphic_grad.

### 3. Comparison code <a class="anchor" id="chapter3"></a>

Code taken from the pennylane discussion. I try to first make this work, and then understand the bugs in my code comparing it to mine.

In [20]:
import pennylane as qml
from pennylane.templates import AngleEmbedding, StronglyEntanglingLayers
from pennylane import numpy as np
import scipy as sp

n_qubits = 3
batch_size = 2

weights = qml.init.strong_ent_layers_normal(n_wires=n_qubits, n_layers=3)
bias = 0.5

theta = (weights, bias)
X = np.random.random(size=[batch_size, n_qubits])
Y = np.random.random(size=[batch_size])


dev = qml.device("default.qubit", wires=n_qubits)
dev.operations.remove("Rot")


@qml.qnode(dev)
def circuit(weights, x):
    """Variational quantum circuit"""
    AngleEmbedding(x, wires=range(n_qubits))
    StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return qml.expval(qml.PauliZ(0))


quantum_grad = qml.grad(circuit, argnum=0)
print("Circuit evaluation:", circuit(weights, X[0]))
print("Gradient evaluation:", quantum_grad(weights, X[0]))


def variational_classifier(theta, x):
    """Variational quantum classifier"""
    weights = theta[0]
    bias = theta[1]
    return circuit(weights, x) + bias


def cost(theta, X, expectations):
    """Cost function for the variational quantum classifier"""
    e_predicted = np.array([variational_classifier(theta, x) for x in X])
    loss = np.mean((e_predicted - expectations) ** 2)
    return loss


print("Cost evaluation:", cost(theta, X, Y))


def cost_ng(theta, X, expectations):
    """Natural gradient of the cost function"""
    weights = theta[0]
    bias = theta[1]

    qnatgrad = np.empty((batch_size,) + weights.shape)
    e_predicted = np.empty([batch_size])

    for idx, x in enumerate(X):
        e_predicted[idx] = variational_classifier(theta, x)
        
        print(x.dtype, x)
        print(weights.dtype, weights)

        # compute the gradient of each QNode with repsect to `weights`
        qgrad = quantum_grad(weights, x)

        # compute the metric tensor of each QNode with respect to `weights`
        num_params = np.prod(qgrad.shape)
        g = circuit.metric_tensor([weights, x])[:num_params, :num_params]

        # compute g^{-1} \nabla U, and reshape it so it has the same shape as `weights`/`qgrad`
        qnatgrad[idx] = np.linalg.solve(g, qgrad.flatten()).reshape(*qgrad.shape)

    # Take the tensordot between the natural gradient and the loss,
    # and divide by the batch size (i.e., taking the mean).
    loss_ng = np.tensordot(2 * (e_predicted - expectations), qnatgrad, axes=1) / batch_size
    return loss_ng


cost_grad = qml.grad(cost, argnum=0)
print("Cost gradient:", cost_grad(theta, X, Y))
print("Cost natural gradient:", cost_ng(theta, X, Y))

<ipython-input-20-4c9f8f69bab5>:9: UserWarning: The init module will be deprecated soon, since templates can now provide a method that returns the shape of parameter tensors.
  weights = qml.init.strong_ent_layers_normal(n_wires=n_qubits, n_layers=3)
/Users/alessandro/opt/anaconda3/lib/python3.8/site-packages/pennylane/qnode.py:755: UserWarning: The QNode.metric_tensor method has been deprecated. Please use the qml.metric_tensor transform instead.
  warnings.warn(
/Users/alessandro/opt/anaconda3/lib/python3.8/site-packages/autograd/numpy/numpy_wrapper.py:77: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return _np.array(args, *array_args, **array_kwargs)


Circuit evaluation: 0.659917798365038
Gradient evaluation: [[[-2.82383286e-02  6.20357635e-02  4.86061353e-05]
  [ 4.86080496e-03  2.49220891e-02  1.35578412e-02]
  [ 8.27616773e-05  1.76661976e-03  1.52114969e-04]]

 [[ 1.39460856e-02  1.13923766e-02  2.39830574e-02]
  [ 2.41788763e-02 -2.29730655e-02  2.41615789e-02]
  [ 2.42046412e-02  1.79156764e-03  2.40034693e-02]]

 [[ 3.98986399e-17  0.00000000e+00  4.16333634e-17]
  [ 2.39549386e-02 -1.92856245e-02  2.77555756e-17]
  [ 2.73536356e-04 -7.16508789e-02  2.77555756e-17]]]
Cost evaluation: 0.19946544258592827
Cost gradient: (array([[[-2.57783756e-02,  4.22319389e-02, -4.21020502e-03],
        [ 7.92981994e-03,  1.74379232e-02,  3.28446985e-02],
        [ 9.42700672e-07,  6.70674968e-04, -1.25659129e-04]],

       [[ 3.07326507e-02,  9.16593276e-03,  3.88823690e-02],
        [ 4.64303602e-02, -1.49282051e-02,  4.75777614e-02],
        [ 4.84616329e-02,  8.26540554e-03,  4.81394543e-02]],

       [[-1.17961196e-16, -1.64798730e-17, -

ValueError: could not broadcast input array from shape (3,3,3) into shape (3)

In [ ]:
x = torch.zeros(2, 1)
print(x)
x.squeeze(1)